In [1]:
ls

drive/  sample_data/


In [2]:
%cd drive/My Drive/Colab Notebooks/rag_nvidia/

/content/drive/My Drive/Colab Notebooks/rag_nvidia


In [3]:
ls

Untitled0.ipynb


In [4]:
!pip install langchain_nvidia_ai_endpoints


In [5]:
!pip install langchain-community langchain-text-splitters faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.4 MB/s eta 0:00:00


In [6]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
from google.colab import userdata
import os
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [13]:

embeddings = NVIDIAEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [16]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


model = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

In [17]:

model

ChatNVIDIA(model='mistralai/mistral-7b-instruct-v0.2')

In [18]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [19]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)


In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [21]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s


In [22]:

for s in final_chain.stream("how can langsmith help with testing"):
    print(s, end="")

 Langsmith is a platform that assists in LLM (Langchain Model) application development, monitoring, and testing. It provides several features to facilitate testing, including:

1. Creating datasets: Developers can create datasets, which are collections of inputs and reference outputs, to be used for running tests on LLM applications.
2. Automated test case curation: Running annotations on traces are critical for test case curation, helping track regressions and improvements, and the development of automatic evaluations.
3. Side-by-side comparison: Langsmith allows users to view different versions of their applications side-by-side within each chart, making it easier to compare performance during A/B testing.
4. Automations: This feature enables users to perform actions on traces in near real-time, such as automatically scoring traces, sending them to annotation queues, or exporting them to datasets.
5. Uploading and running test cases: Langsmith makes it easy to upload, create, or expo

In [25]:
# prompt: openai

!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [28]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama3-8b-instruct",
  messages=[{"role":"user","content":"Provide an essay of Machine Learning and Gen AI"}],
  temperature=0.5,
  top_p=1,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



**The Convergence of Machine Learning and General Artificial Intelligence: Unlocking the Future of Human-Machine Collaboration**

The intersection of machine learning (ML) and general artificial intelligence (Gen AI) has given rise to a new era of human-machine collaboration, revolutionizing the way we live, work, and interact with each other. Machine learning, a subset of artificial intelligence, enables machines to learn from data without being explicitly programmed. General artificial intelligence, on the other hand, refers to the development of machines that possess human-like intelligence, capable of performing any intellectual task that a human can. The convergence of these two fields has far-reaching implications, transforming industries, and transforming the way we live.

**Machine Learning: The Foundation of AI**

Machine learning is a subset of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data. This approa

In [ ]:
langchain